# Generating Image Overlays of Wx Variables

First obtain the maximum and minimum values of Latitude and Longitude so that the image can be placed in Leaflet.

In [13]:
lat_min = float(lats.min())
lat_max = float(lats.max())
lon_min = float(lons.min())
lon_max = float(lons.max())

print(f"Image bounds: SW=({lat_min}, {lon_min}), NE=({lat_max}, {lon_max})")


Image bounds: SW=(-3.236724853515625, 44.73521041870117), NE=(43.468475341796875, 111.26478576660156)


Lets generate the images.

## Temperature

In [22]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
from matplotlib.colors import LinearSegmentedColormap
from scipy.ndimage import gaussian_filter
from tqdm import notebook


# Load WRF data and Calculate Wind Speed
ds = xr.open_dataset("/Users/manaruchi/Desktop/WeatherDataViz/raw_data/AFCNWP_WRF_model_output_00UTC.nc")

# Generate hourly images for 3 days
tlimit = 72

for t in notebook.tqdm(range(tlimit), desc="Generating Hourly Surface Temperature Images.."):
    t2 = ds["T2"].isel(Time=t) - 273.15
    lats = ds["XLAT"].isel(Time=t).values
    lons = ds["XLONG"].isel(Time=t).values
    
    # Smooth Temperature with Gaussian filter (sigma controls smoothness)
    t2_smooth = gaussian_filter(t2, sigma=1)

    # Windy color scale definition - RdBu_r
    windy_colors = [
          "#67001f",  # Dark Red
          "#b2182b",  # Red
          "#d6604d",  # Light Red
          "#f4a582",  # Peach
          "#fddbc7",  # Light Pink
          "#ffffff",  # White (neutral/midpoint)
          "#d1e5f0",  # Light Blue
          "#92c5de",  # Sky Blue
          "#4393c3",  # Medium Blue
          "#2166ac",  # Blue
          "#053061"   # Dark Blue
        ]

    
    windy_cmap = LinearSegmentedColormap.from_list("windy", windy_colors[::-1])
    
    # Plot
    fig = plt.figure(figsize=(20, 16))
    ax = plt.axes(projection=ccrs.PlateCarree())
    
    ax.axis("off")
    ax.set_frame_on(False)
    fig.patch.set_alpha(0.0)
    
    # Compute image extent for imshow: [left, right, bottom, top]
    extent = [lons.min(), lons.max(), lats.min(), lats.max()]
    
    # Use imshow with interpolation for smooth result
    img = ax.imshow(
        t2_smooth,
        origin='lower',
        extent=extent,
        cmap=windy_cmap,
        vmin=-10,
        vmax=50,
        interpolation='bilinear',   # 'bilinear' or 'bicubic' for smoothness
        alpha=0.8
    )
    
    ax.set_extent(extent, crs=ccrs.PlateCarree())
    
    plt.savefig(
        f"temperature_{t}_0.png",
        dpi=300,
        bbox_inches='tight',
        pad_inches=0,
        transparent=True
    )
    plt.close()

Generating Hourly Surface Temperature Images..:   0%|          | 0/72 [00:00<?, ?it/s]

In [3]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
from matplotlib.colors import LinearSegmentedColormap
from scipy.ndimage import gaussian_filter
from tqdm import notebook
ds = xr.open_dataset("/Users/manaruchi/Desktop/WeatherDataViz/raw_data/AFCNWP_WRF_model_output_00UTC.nc")

In [4]:
ds

<xarray.Dataset> Size: 18GB
Dimensions:                (Time: 124, south_north: 198, west_east: 255,
                            bottom_top: 32, bottom_top_stag: 33,
                            soil_layers_stag: 4, west_east_stag: 256,
                            south_north_stag: 199, pft_fraction_stag: 17,
                            seed_dim_stag: 2)
Coordinates:
    XLAT                   (Time, south_north, west_east) float32 25MB ...
    XLONG                  (Time, south_north, west_east) float32 25MB ...
    XTIME                  (Time) datetime64[ns] 992B ...
    XLAT_U                 (Time, south_north, west_east_stag) float32 25MB ...
    XLONG_U                (Time, south_north, west_east_stag) float32 25MB ...
    XLAT_V                 (Time, south_north_stag, west_east) float32 25MB ...
    XLONG_V                (Time, south_north_stag, west_east) float32 25MB ...
Dimensions without coordinates: Time, south_north, west_east, bottom_top,
                                bottom_top_stag, soil_layers_stag,
                                west_east_stag, south_north_stag,
                                pft_fraction_stag, seed_dim_stag
Data variables: (12/159)
    Times                  (Time) |S19 2kB ...
    LU_INDEX               (Time, south_north, west_east) float32 25MB ...
    ZNU                    (Time, bottom_top) float32 16kB ...
    ZNW                    (Time, bottom_top_stag) float32 16kB ...
    ZS                     (Time, soil_layers_stag) float32 2kB ...
    DZS                    (Time, soil_layers_stag) float32 2kB ...
    ...                     ...
    PCB                    (Time, south_north, west_east) float32 25MB ...
    PC                     (Time, south_north, west_east) float32 25MB ...
    LANDMASK               (Time, south_north, west_east) float32 25MB ...
    LAKEMASK               (Time, south_north, west_east) float32 25MB ...
    SST                    (Time, south_north, west_east) float32 25MB ...
    SST_INPUT              (Time, south_north, west_east) float32 25MB ...
Attributes: (12/134)
    TITLE:                            OUTPUT FROM WRF V4.3 MODEL
    START_DATE:                      2025-02-19_00:00:00
    SIMULATION_START_DATE:           2025-02-19_00:00:00
    WEST-EAST_GRID_DIMENSION:        256
    SOUTH-NORTH_GRID_DIMENSION:      199
    BOTTOM-TOP_GRID_DIMENSION:       33
    ...                              ...
    ISLAKE:                          -1
    ISICE:                           24
    ISURBAN:                         1
    ISOILWATER:                      14
    HYBRID_OPT:                      2
    ETAC:                            0.2

# Vertical Temperature

Lets first check the extent of the values in each level.

In [18]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
from matplotlib.colors import LinearSegmentedColormap
from scipy.ndimage import gaussian_filter
from tqdm import notebook

# Load WRF NetCDF file
ds = xr.open_dataset("/Users/manaruchi/Desktop/WeatherDataViz/raw_data/AFCNWP_WRF_model_output_00UTC.nc")
print("Dataset Loaded...")
tlimit = 72 # hourly data for 3 days (24x3)

levels = [0,6,11,14,16,21,24,28,31] # Levels in concurrence with bottom_top values


for i,level in enumerate(levels):

        
    potT = ds["T"].isel(Time=0, bottom_top=level)
    perP = ds["P"].isel(Time=0, bottom_top=level)
    baseP = ds["PB"].isel(Time=0, bottom_top=level)
    lats = ds["XLAT"].isel(Time=0).values
    lons = ds["XLONG"].isel(Time=0).values

    # Compute Pressure in Pa → convert to hPa
    pressure = (perP + baseP) / 100.0 

    # Compute potential temperature: theta = T + 300
    theta = potT + 300.0 # In K

    # Convert to actual temperature using Poisson's equation
    T_actual = theta * (pressure / 1000.0) ** 0.2854  # in K
    T_actual = T_actual - 273.15 # In degree C
    
    print(i+1, np.min(T_actual.values), np.max(T_actual.values))

Dataset Loaded...
1 -35.28708 28.428375
2 -31.118774 24.98819
3 -31.220901 16.155762
4 -33.901962 7.0884705
5 -37.34262 -0.4804077
6 -56.413467 -27.026627
7 -61.073242 -42.78546
8 -81.660645 -52.15953
9 -72.47908 -49.88983


In [24]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
from matplotlib.colors import LinearSegmentedColormap
from scipy.ndimage import gaussian_filter
from tqdm import notebook

# Load WRF NetCDF file
ds = xr.open_dataset("/Users/manaruchi/Desktop/WeatherDataViz/raw_data/AFCNWP_WRF_model_output_00UTC.nc")
print("Dataset Loaded...")
tlimit = 72 # hourly data for 3 days (24x3)

levels = [0,6,11,14,16,21,24,28,31] # Levels in concurrence with bottom_top values
vmin_values = [-10,-20,-30,-40,-40,-60,-70,-80,-80] # VMin values for different layers
vmax_values = [40,30,20,10,10,0,-10,-30,-40] # VMax values for different layers


for i,level in enumerate(levels):
    for t in notebook.tqdm(range(tlimit), desc=f"Generating Hourly Temperature Image. Level: {i+1} Min Value: {vmin_values[i]} Max Value: {vmax_values[i]}"):
        potT = ds["T"].isel(Time=t, bottom_top=level)
        perP = ds["P"].isel(Time=t, bottom_top=level)
        baseP = ds["PB"].isel(Time=t, bottom_top=level)
        lats = ds["XLAT"].isel(Time=t).values
        lons = ds["XLONG"].isel(Time=t).values

        # Compute Pressure in Pa → convert to hPa
        pressure = (perP + baseP) / 100.0 

        # Compute potential temperature: theta = T + 300
        theta = potT + 300.0 # In K

        # Convert to actual temperature using Poisson's equation
        T_actual = theta * (pressure / 1000.0) ** 0.2854  # in K
        T_actual = T_actual - 273.15 # In degree C
        
        # Smooth wind speed with Gaussian filter (sigma controls smoothness)
        t_smooth = gaussian_filter(T_actual, sigma=1)
        
        # Windy color scale definition
        if(i<2):
            
            # Windy color scale definition - RdBu_r
            windy_colors = [
              "#053061",  # Dark Blue
              "#2166ac",  # Blue
              "#4393c3",  # Medium Blue
              "#92c5de",  # Sky Blue
              "#d1e5f0",  # Light Blue
              "#ffffff",  # White (neutral/midpoint)
              "#fddbc7",  # Light Pink
              "#f4a582",  # Peach
              "#d6604d",  # Light Red
              "#b2182b",  # Red
              "#67001f"   # Dark Red
            ]

        else:
            windy_colors = [
              "#021025",  # Near-black blue
              "#041f4a",  # Very dark blue
              "#08306b",  # Dark navy blue
              "#08519c",  # Strong blue
              "#2171b5",  # Deep sky blue
              "#4292c6",  # Medium blue
              "#6baed6",  # Medium light blue
              "#9ecae1",  # Sky blue
              "#c6dbef",  # Soft blue
              "#deebf7",  # Pale blue
              "#f7fbff"   # Very light blue (almost white)
            ]

        windy_cmap = LinearSegmentedColormap.from_list("windy", windy_colors)
        
        # Plot
        fig = plt.figure(figsize=(20, 16))
        ax = plt.axes(projection=ccrs.PlateCarree())
        
        ax.axis("off")
        ax.set_frame_on(False)
        fig.patch.set_alpha(0.0)
        
        # Compute image extent for imshow: [left, right, bottom, top]
        extent = [lons.min(), lons.max(), lats.min(), lats.max()]

        
        # Use imshow with interpolation for smooth result
        img = ax.imshow(
            t_smooth,
            origin='lower',
            extent=extent,
            cmap=windy_cmap,
            vmin=vmin_values[i],
            vmax=vmax_values[i],
            interpolation='bilinear',   # 'bilinear' or 'bicubic' for smoothness
            alpha=0.8
        )
        
        ax.set_extent(extent, crs=ccrs.PlateCarree())
        
        plt.savefig(
            f"temperature_{t}_{i+1}.png",
            dpi=300,
            bbox_inches='tight',
            pad_inches=0,
            transparent=True
        )
        plt.close()


Dataset Loaded...


Generating Hourly Temperature Image. Level: 1 Min Value: -10 Max Value: 40:   0%|          | 0/72 [00:00<?, ?i…

Generating Hourly Temperature Image. Level: 2 Min Value: -20 Max Value: 30:   0%|          | 0/72 [00:00<?, ?i…

Generating Hourly Temperature Image. Level: 3 Min Value: -30 Max Value: 20:   0%|          | 0/72 [00:00<?, ?i…

Generating Hourly Temperature Image. Level: 4 Min Value: -40 Max Value: 10:   0%|          | 0/72 [00:00<?, ?i…

Generating Hourly Temperature Image. Level: 5 Min Value: -40 Max Value: 10:   0%|          | 0/72 [00:00<?, ?i…

Generating Hourly Temperature Image. Level: 6 Min Value: -60 Max Value: 0:   0%|          | 0/72 [00:00<?, ?it…

Generating Hourly Temperature Image. Level: 7 Min Value: -70 Max Value: -10:   0%|          | 0/72 [00:00<?, ?…

Generating Hourly Temperature Image. Level: 8 Min Value: -80 Max Value: -30:   0%|          | 0/72 [00:00<?, ?…

Generating Hourly Temperature Image. Level: 9 Min Value: -80 Max Value: -40:   0%|          | 0/72 [00:00<?, ?…